# Reduce

Break up the full set into smaller groups, e.g. by zip code, and save a separate files to make analysis with the data more manageable

In [1]:
import s3fs 
import pandas as pd
import boto3
from io import StringIO
import numpy as np

### Sessions

In [2]:
sessions = pd.read_csv('s3://script.chargepoint.data/Residential_Data/clean/sessions_paired.csv')
sessions

,Session ID,Station ID,Port ID,Connector Type,POI Category,POI Subcategory,Station Start Time (Local),Station End Time (Local),Session Time Zone,Session Time (secs),...,Driver Country,Max Power,Session Type,Vehicle Model Year,start_datetime,start_seconds,start_year,start_month,start_day,start_weekday
0,26957013,77863,87501,J1772,Single family residential,NaN,2015-05-26 17:05:01.0,2015-05-27 06:35:39.0,PDT,48637,...,United States,6.6,OTHER,NaN,2015-05-26 17:05:01,61501,2015,5,145,1
1,26995359,8911,138493,J1772,Single family residential,NaN,2015-05-27 18:00:26.0,2015-05-28 17:41:31.0,PDT,85264,...,United States,6.6,OTHER,NaN,2015-05-27 18:00:26,64826,2015,5,146,2
2,27001907,76933,87301,J1772,Single family residential,NaN,2015-05-27 19:28:18.0,2015-05-28 14:22:19.0,PDT,68041,...,United States,6.6,OTHER,NaN,2015-05-27 19:28:18,70098,2015,5,146,2
3,26999743,7200,137921,J1772,Single family residential,NaN,2015-05-27 22:03:00.0,2015-05-30 09:25:31.0,PDT,213750,...,United States,6.6,OTHER,NaN,2015-05-27 22:03:00,79380,2015,5,146,2
4,27033203,3564,145229,J1772,Single family residential,NaN,2015-05-28 14:23:26.0,2015-05-28 20:20:05.0,PDT,21399,...,United States,6.6,OTHER,2012.0,2015-05-28 14:23:26,51806,2015,5,147,3
5,27034565,70673,159037,J1772,Single family residential,NaN,2015-05-28 14:58:10.0,2015-05-28 17:04:33.0,PDT,7583,...,United States,6.6,OTHER,NaN,2015-05-28 14:58:10,53890,2015,5,147,3
6,27037151,78553,87733,J1772,Single family residential,NaN,2015-05-28 16:07:03.0,2015-05-28 19:19:09.0,PDT,11531,...,United States,6.6,OTHER,NaN,2015-05-28 16:07:03,58023,2015,5,147,3
7,27038759,76973,87309,J1772,Single family residential,NaN,2015-05-28 16:57:58.0,2015-05-29 07:35:47.0,PDT,52669,...,United States,6.6,OTHER,NaN,2015-05-28 16:57:58,61078,2015,5,147,3
8,27039769,77603,87461,J1772,Single family residential,NaN,2015-05-28 17:30:55.0,2015-05-29 06:06:02.0,PDT,45307,...,United States,6.6,OTHER,NaN,2015-05-28 17:30:55,63055,2015,5,147,3
9,27040409,77883,87505,J1772,Single family residential,NaN,2015-05-28 17:54:45.0,2015-05-28 20:49:28.0,PDT,10483,...,United States,6.6,OTHER,NaN,2015-05-28 17:54:45,64485,2015,5,147,3


In [3]:
sessions.columns

Index(['Session ID', 'Station ID', 'Port ID', 'Connector Type', 'POI Category',
       'POI Subcategory', 'Station Start Time (Local)',
       'Station End Time (Local)', 'Session Time Zone', 'Session Time (secs)',
       'Charging Time (secs)', 'Start SoC', 'End SoC', 'Full Charge',
       'Energy (kWh)', 'Driver ID', 'Driver Zip', 'Vehicle Make',
       'Vehicle Model', 'Battery Capacity', 'EV Type', '$ Fee', 'Fee Currency',
       'Zip Code', 'City', 'County', 'State', 'Country', 'Driver City',
       'Driver State', 'Driver Country', 'Max Power', 'Session Type',
       'Vehicle Model Year', 'start_datetime', 'start_seconds', 'start_year',
       'start_month', 'start_day', 'start_weekday'],
      dtype='object')

In [20]:
sessions.loc[10, 'Zip Code'][:5]

'94510'

In [35]:
sessions[sessions['Zip Code'].str.len() > 5]['Zip Code'].index

Int64Index([  91010,   91384,   91607,   91693,   91718,   92642,   92827,
              92947,   93229,   93293,
            ...
            1025733, 1025788, 1025845, 1025952, 1026338, 1026507, 1026631,
            1027054, 1027072, 1027368],
           dtype='int64', length=3461)

In [36]:
for i in sessions[sessions['Zip Code'].str.len() > 5]['Zip Code'].index:
    sessions.loc[i, 'Zip Code'] = sessions.loc[i, 'Zip Code'][:5]

In [38]:
max(sessions['Zip Code'].str.len())

5

In [43]:
csv_buffer = StringIO()
sessions.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
key = 's3://script.chargepoint.data/Residential_Data/clean/sessions_paired.csv'
s3_resource.Object('script.chargepoint.data', key).put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'FFF2D24422176F25',
  'HostId': 'yc9XInakoMVkKC8fBENljKBUgig6DP9sfeSaQ+wt19nbRpr744Kn20HaM6yWeBV8uIzvbGShkhs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'yc9XInakoMVkKC8fBENljKBUgig6DP9sfeSaQ+wt19nbRpr744Kn20HaM6yWeBV8uIzvbGShkhs=',
   'x-amz-request-id': 'FFF2D24422176F25',
   'date': 'Sat, 27 Jul 2019 22:09:10 GMT',
   'etag': '"de2b7ee193351c68b55fa13f30b9fc70"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"de2b7ee193351c68b55fa13f30b9fc70"'}

In [39]:
zip_code_list = list(set(sessions['Zip Code']))
len(zip_code_list)

246

In [40]:
zip_code_list[0]

'94510'

In [41]:
session_ids = {}
ct = 0
for zip_code in zip_code_list: 
    if np.mod(ct, 10) == 0:
        print('On zip_code ',ct,' of ', len(zip_code_list))
        
    df = sessions[sessions['Zip Code'] == zip_code]
    session_ids[zip_code] = df['Session ID'].values
    
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3_resource = boto3.resource('s3')
    key = 'Residential_Data/clean/Reduced/By_Zipcode/sessions_zipcode'+str(zip_code)+'.csv'
    s3_resource.Object('script.chargepoint.data', key).put(Body=csv_buffer.getvalue())
    
    ct += 1

On zip_code  0  of  246
On zip_code  10  of  246
On zip_code  20  of  246
On zip_code  30  of  246
On zip_code  40  of  246
On zip_code  50  of  246
On zip_code  60  of  246
On zip_code  70  of  246
On zip_code  80  of  246
On zip_code  90  of  246
On zip_code  100  of  246
On zip_code  110  of  246
On zip_code  120  of  246
On zip_code  130  of  246
On zip_code  140  of  246
On zip_code  150  of  246
On zip_code  160  of  246
On zip_code  170  of  246
On zip_code  180  of  246
On zip_code  190  of  246
On zip_code  200  of  246
On zip_code  210  of  246
On zip_code  220  of  246
On zip_code  230  of  246
On zip_code  240  of  246


In [9]:
intervals = pd.read_csv('s3://script.chargepoint.data/Residential_Data/clean/intervals_paired.csv')

In [12]:
intervals.columns

Index(['Interval ID', 'Session ID', 'Interval Start Time (Local)',
       'Interval Duration (Secs)', 'Interval Energy', 'Peak Power',
       'Average Power'],
      dtype='object')

In [42]:
ct = 0
for zip_code in zip_code_list: 
    if np.mod(ct, 10) == 0:
        print('On zip_code ',ct,' of ', len(zip_code_list))
        
    ids = session_ids[zip_code]
    df = intervals[intervals['Session ID'].isin(ids)]

    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3_resource = boto3.resource('s3')
    key = 'Residential_Data/clean/Reduced/By_Zipcode/intervals_zipcode'+str(zip_code)+'.csv'
    s3_resource.Object('script.chargepoint.data', key).put(Body=csv_buffer.getvalue())
    
    ct += 1

On zip_code  0  of  246
On zip_code  10  of  246
On zip_code  20  of  246
On zip_code  30  of  246
On zip_code  40  of  246
On zip_code  50  of  246
On zip_code  60  of  246
On zip_code  70  of  246
On zip_code  80  of  246
On zip_code  90  of  246
On zip_code  100  of  246
On zip_code  110  of  246
On zip_code  120  of  246
On zip_code  130  of  246
On zip_code  140  of  246
On zip_code  150  of  246
On zip_code  160  of  246
On zip_code  170  of  246
On zip_code  180  of  246
On zip_code  190  of  246
On zip_code  200  of  246
On zip_code  210  of  246
On zip_code  220  of  246
On zip_code  230  of  246
On zip_code  240  of  246
